# Data2Text

Make a start on some report generation.

In [4]:
import notebookimport

In [5]:
sr = __import__("Charts - Stage Results")
ssd = __import__("Charts - Split Sector Delta")
sp = __import__("Charts - Stage Progress")

importing Jupyter notebook from Charts - Stage Results.ipynb
importing Jupyter notebook from Charts - Split Sector Delta.ipynb
importing Jupyter notebook from Charts - Stage Progress.ipynb


In [1]:
import tracery
from tracery.modifiers import base_english
from inflectenglish import inflect_english
from pytracery_logic import pytracery_logic

def pandas_row_mapper(row, rules, root,  modifiers=base_english):
    ''' Function to parse single row of dataframe '''
    row=row.to_dict()
    rules=rules.copy()

    for k in row:
        rules[k] = str(row[k])
        
    grammar = tracery.Grammar(rules)
        
    if modifiers is not None:
        if isinstance(modifiers,list):
            for modifier in modifiers:
                grammar.add_modifiers(modifier)
        else:
            grammar.add_modifiers(modifiers)

    return grammar.flatten(root)

In [2]:
import os
import sqlite3
import pandas as pd

#dbname='wrc18.db'
#dbname='france18.db'
#conn = sqlite3.connect(dbname)

if __name__=='__main__':
    #dbname='wrc18.db'
    dbname='spain18.db'
    conn = sqlite3.connect(dbname)
    rally='Spain'
    rc='RC1'
    year=2018
    #ss='SS4'

In [59]:
import inflect
p = inflect.engine()

def timedelta_to_text(td): 
    if not td: return
    c=pd.to_timedelta(td).components
    txt=[]
    if c.hours>0:
        txt_.append(p.number_to_words(c.hours)+p.plural('hours',c.hours))
    if c.minutes>0:
        txt.append('{} {}'.format(p.number_to_words(c.minutes),p.plural('minute',c.minutes)))
    seconds=round(c.seconds+c.milliseconds/1000, 2)
    txt.append('{} {}'.format(p.number_to_words(seconds),p.plural('second',seconds)))
    return p.join(txt, final_sep="and")

def timedelta_to_numtext(td):
    if not td: return
    c=pd.to_timedelta(td).components
    txt=[]
    if c.hours>0:
        txt.append('{}{}'.format(c.hours,p.plural('hr',c.hours)))
    if c.minutes>0:
        txt.append('{}m'.format(c.minutes))
    #if c.seconds>0:
    #    seconds=c.seconds
    #    if c.milliseconds>0:
    #        seconds=round(seconds+c.milliseconds/1000, 2)
    seconds=round(c.seconds+c.milliseconds/1000, 2)
    txt.append('{}s'.format(seconds))
    return p.join(txt,conj='')

In [69]:
#https://stackoverflow.com/a/17847006/454773
from string import Formatter

def strfdelta(tdelta, fmt="{H:02}:{M:02}:{S:02.1f}"):
    f = Formatter()
    d = {}
    l = {'D': 86400, 'H': 3600, 'M': 60, 'S': 1}
    k = list(map( lambda x: x[1], list(f.parse(fmt))))
    rem = int(tdelta.total_seconds())

    for i in ('D', 'H', 'M', 'S'):
        if i in k and i in l.keys():
            d[i], rem = divmod(rem, l[i])
    d['S']=d['S']+tdelta.microseconds/1000000
    return f.format(fmt, **d)

In [68]:
import re
import isodate
import pandas as pd

def interval(timestring):
    ''' Try to return a commonly formatted time string. '''
    if timestring.startswith('PT'):
        return strfdelta(isodate.parse_duration(timestring))
    elif re.match(r"^[0-9]*:[0-9]*:[0-9\.]*$", timestring):
        return strfdelta(pd.to_timedelta(timestring))
    
    return timestring


#Then maybe register soemthing with sqlite as part of the data ingest?
#eg https://stackoverflow.com/a/27641141/454773 ?


interval('00:04:45.5000000'), interval('PT10M40.4S'), interval('PT1H9M30.8S'), interval('PT30.8S')

('00:04:45.5', '00:10:40.4', '01:09:30.8', '00:00:30.8')

In [70]:

strfdelta(pd.to_timedelta('00:04:45.3000000'))

'00:04:45.3'

In [20]:
import isodate
strfdelta(pd.to_timedelta(isodate.parse_duration('PT1H5M26S')), "%H:%M:%S")

'1:5:26'

In [14]:
timedelta_to_numtext(pd.to_timedelta('00:04:45.5000000'))

'4m 45.5s'

In [6]:
stageresults = sr.dbGetRallyStages(conn, rally).sort_values('number')

#The getEnrichedStageRank query filters down to a class - default is RC1
stagerank_overall = sr.getEnrichedStageRank(conn, rally, typ='overall')

stagerank_stage = sr.getEnrichedStageRank(conn, rally, typ='stage')

In [52]:
stagerank_stage.head()

,diffFirst,diffFirstMs,diffPrev,diffPrevMs,elapsedDuration,elapsedDurationMs,entryId,position,source,stageId,...,retainedClassLead,gainedOverallPos,gainedOverallLead,overallPosDiff,lostOverallLead,retainedOverallLead,stagewin,stagewincount,winsinarow,gainedTime
0,00:00:00,0.0,00:00:00,0.0,00:03:35.3000000,215300.0,3040,1.0,Default,823,...,False,False,False,0.0,False,False,True,1.0,1,False
1,00:00:03.7000000,3700.0,00:00:03.7000000,3700.0,00:03:39,219000.0,3044,2.0,Default,823,...,False,False,False,0.0,False,False,False,0.0,0,False
2,00:00:04.2000000,4200.0,00:00:00.5000000,500.0,00:03:39.5000000,219500.0,3047,3.0,Default,823,...,False,False,False,0.0,False,False,False,0.0,0,False
3,00:00:05.6000000,5600.0,00:00:01.4000000,1400.0,00:03:40.9000000,220900.0,3043,4.0,Default,823,...,False,False,False,0.0,False,False,False,0.0,0,False
4,00:00:06,6000.0,00:00:00.4000000,400.0,00:03:41.3000000,221300.0,3041,5.0,Default,823,...,False,False,False,0.0,False,False,False,0.0,0,False


In [7]:
stageresults.head()

,legDate,date,startListId,status,itineraryLegId,itinerarySectionId,section,order,code,distance,eventId,name,number,stageId,stageType,status,timingPrecision,itineraryLegId,itinerarySections.itinerarySectionId
0,2018-10-25,Thursday 25th October,236.0,Completed,143,328,Section 2,2,SS1,3.20,37,Barcelona (asphalt) TV Live,1,823,SpecialStage,Completed,Tenth,143,328
1,2018-10-26,Friday 26th October,234.0,Completed,142,329,Section 3,3,SS2,7.00,37,Gandesa 1 (gravel),2,828,SpecialStage,Completed,Tenth,142,329
2,2018-10-26,Friday 26th October,234.0,Completed,142,329,Section 3,3,SS3,26.59,37,Pesells 1 (gravel),3,830,SpecialStage,Completed,Tenth,142,329
3,2018-10-26,Friday 26th October,234.0,Completed,142,329,Section 3,3,SS4,38.85,37,La Fatarella -Vilalba 1 (gravel & asphalt),4,831,SpecialStage,Interrupted,Tenth,142,329
6,2018-10-26,Friday 26th October,234.0,Completed,142,330,Section 4,4,SS5,7.00,37,Gandesa 2 (gravel),5,835,SpecialStage,Completed,Tenth,142,330


In [137]:
rules = {'onetwo_':'[branch:#_onetwo_#]#branch##condition#',
         '_onetwo_':'#stagewin._branch(True,stagefirst,stagesecond)# ',
         'y':'[switch:#sval#]#switch##condition#',
         'yy':'#drivercode# [switch:#position.int_to_words.ordinal._prefix(stage)._switch#]#switch##condition#',
         'sval':'#position.int_to_words.ordinal._prefix(stage)._switch#',
         'stagefirst':'#stagewinner_#',
         'stagethird':'took third, a further #diffPrev_txt# #backbehind# (#diffFirst_txt# off the lead) in #elapsedDuration_txt#',
         'stagewinner_': "#won_# stage #code# #inwith_# a time of #elapsedDuration_txt#.",
         'won_':['won', 'took'],
         #'keeploselead_':'[branch:#_keeploselead_#]#branch##condition#','_keeploselead_':'#retainedOverallLead._branch(True,retlead,lostlead)#',
         #'retlead':'retaining the overall lead', 'lostlead':'losing the overall lead',
         'inwith_':['in', 'with'], 'backbehind':['back','behind'],
        'stagesecond':'took second, #diffFirst_txt# behind in #elapsedDuration_txt#.',
        'z':'#position._gte(1,stagewinner)#',
        'toptwo_':'[switch:#_toptwo_#]#switch##condition#','_toptwo_':'#position._gte(2,onetwo_)#',
        'null':''}

In [131]:
_code='SS5'
typ=stagerank_stage
typ['elapsedDuration_txt'] =typ['elapsedDuration'].apply(timedelta_to_numtext)
typ['diffFirst_txt'] =typ['diffFirst'].apply(timedelta_to_numtext)
typ['diffPrev_txt'] =typ['diffPrev'].apply(timedelta_to_numtext)
typ.head()

,diffFirst,diffFirstMs,diffPrev,diffPrevMs,elapsedDuration,elapsedDurationMs,entryId,position,source,stageId,...,overallPosDiff,lostOverallLead,retainedOverallLead,stagewin,stagewincount,winsinarow,gainedTime,elapsedDuration_txt,diffFirst_txt,diffPrev_txt
0,00:00:00,0.0,00:00:00,0.0,00:03:35.3000000,215300.0,3040,1.0,Default,823,...,0.0,False,False,True,1.0,1,False,3m 35.3s,0.0s,0.0s
1,00:00:03.7000000,3700.0,00:00:03.7000000,3700.0,00:03:39,219000.0,3044,2.0,Default,823,...,0.0,False,False,False,0.0,0,False,3m 39.0s,3.7s,3.7s
2,00:00:04.2000000,4200.0,00:00:00.5000000,500.0,00:03:39.5000000,219500.0,3047,3.0,Default,823,...,0.0,False,False,False,0.0,0,False,3m 39.5s,4.2s,0.5s
3,00:00:05.6000000,5600.0,00:00:01.4000000,1400.0,00:03:40.9000000,220900.0,3043,4.0,Default,823,...,0.0,False,False,False,0.0,0,False,3m 40.9s,5.6s,1.4s
4,00:00:06,6000.0,00:00:00.4000000,400.0,00:03:41.3000000,221300.0,3041,5.0,Default,823,...,0.0,False,False,False,0.0,0,False,3m 41.3s,6.0s,0.4s


In [132]:
typ[(typ['code']==_code)]

,diffFirst,diffFirstMs,diffPrev,diffPrevMs,elapsedDuration,elapsedDurationMs,entryId,position,source,stageId,...,overallPosDiff,lostOverallLead,retainedOverallLead,stagewin,stagewincount,winsinarow,gainedTime,elapsedDuration_txt,diffFirst_txt,diffPrev_txt
56,00:00:00,0.0,00:00:00,0.0,00:04:14.7000000,254700.0,3045,1.0,Default,835,...,-6.0,False,False,True,1.0,1,True,4m 14.7s,0.0s,0.0s
57,00:00:00.5000000,500.0,00:00:00.5000000,500.0,00:04:15.2000000,255200.0,3046,2.0,Default,835,...,-9.0,False,False,False,1.0,0,True,4m 15.2s,0.5s,0.5s
58,00:00:00.8000000,800.0,00:00:00.3000000,300.0,00:04:15.5000000,255500.0,3050,3.0,Default,835,...,-5.0,False,False,False,0.0,0,True,4m 15.5s,0.8s,0.3s
59,00:00:01.1000000,1100.0,00:00:00.3000000,300.0,00:04:15.8000000,255800.0,3041,4.0,Default,835,...,0.0,False,False,False,0.0,0,True,4m 15.8s,1.1s,0.3s
60,00:00:01.3000000,1300.0,00:00:00.2000000,200.0,00:04:16,256000.0,3047,5.0,Default,835,...,3.0,False,False,False,1.0,0,False,4m 16.0s,1.3s,0.2s
61,00:00:03.7000000,3700.0,00:00:02.4000000,2400.0,00:04:18.4000000,258400.0,3043,6.0,Default,835,...,5.0,True,False,False,1.0,0,False,4m 18.4s,3.7s,2.4s
62,00:00:03.8000000,3800.0,00:00:00.1000000,100.0,00:04:18.5000000,258500.0,3049,7.0,Default,835,...,4.0,False,False,False,0.0,0,False,4m 18.5s,3.8s,0.1s
63,00:00:03.9000000,3900.0,00:00:00.1000000,100.0,00:04:18.6000000,258600.0,3040,8.0,Default,835,...,2.0,False,False,False,1.0,0,True,4m 18.6s,3.9s,0.1s
64,00:00:04.2000000,4200.0,00:00:00.3000000,300.0,00:04:18.9000000,258900.0,3044,9.0,Default,835,...,0.0,False,False,False,0.0,0,True,4m 18.9s,4.2s,0.3s
65,00:00:05.1000000,5100.0,00:00:00.9000000,900.0,00:04:19.8000000,259800.0,3042,10.0,Default,835,...,0.0,False,False,False,0.0,0,True,4m 19.8s,5.1s,0.9s


In [133]:
typ.columns

Index(['diffFirst', 'diffFirstMs', 'diffPrev', 'diffPrevMs', 'elapsedDuration',
       'elapsedDurationMs', 'entryId', 'position', 'source', 'stageId',
       'stageTimeId', 'status', 'class', 'code', 'distance', 'name', 'snum',
       'driver.code', 'entrant.name', 'classrank', 'gainedClassPos',
       'gainedClassLead', 'classPosDiff', 'lostClassLead', 'retainedClassLead',
       'gainedOverallPos', 'gainedOverallLead', 'overallPosDiff',
       'lostOverallLead', 'retainedOverallLead', 'stagewin', 'stagewincount',
       'winsinarow', 'gainedTime', 'elapsedDuration_txt', 'diffFirst_txt',
       'diffPrev_txt'],
      dtype='object')

In [134]:
_srs=typ[(typ['code']==_code) & (typ['position']<=3)][['code','position','winsinarow','elapsedDuration',
                                                       'diffFirstMs','diffPrevMs','diffPrev_txt',
                                                    'elapsedDuration_txt', 'stagewin','diffFirst_txt',
                                                                                         'stagewincount',
                                                                                         'driver.code']]
_srs.columns=[c.replace('.','') for c in _srs.columns]
_srs.to_dict(orient='record')

[{'code': 'SS5',
  'position': 1.0,
  'winsinarow': 1,
  'elapsedDuration': '00:04:14.7000000',
  'diffFirstMs': 0.0,
  'diffPrevMs': 0.0,
  'diffPrev_txt': '0.0s',
  'elapsedDuration_txt': '4m 14.7s',
  'stagewin': True,
  'diffFirst_txt': '0.0s',
  'stagewincount': 1.0,
  'drivercode': 'SOR'},
 {'code': 'SS5',
  'position': 2.0,
  'winsinarow': 0,
  'elapsedDuration': '00:04:15.2000000',
  'diffFirstMs': 500.0,
  'diffPrevMs': 500.0,
  'diffPrev_txt': '0.5s',
  'elapsedDuration_txt': '4m 15.2s',
  'stagewin': False,
  'diffFirst_txt': '0.5s',
  'stagewincount': 1.0,
  'drivercode': 'LAT'},
 {'code': 'SS5',
  'position': 3.0,
  'winsinarow': 0,
  'elapsedDuration': '00:04:15.5000000',
  'diffFirstMs': 800.0,
  'diffPrevMs': 300.0,
  'diffPrev_txt': '0.3s',
  'elapsedDuration_txt': '4m 15.5s',
  'stagewin': False,
  'diffFirst_txt': '0.8s',
  'stagewincount': 0.0,
  'drivercode': 'BRE'}]

In [135]:
_srs[_srs['position']==1].apply(lambda row: pandas_row_mapper(row, rules, "#stagewinner_#",[base_english,inflect_english,pytracery_logic]), axis=1)


56    won stage SS5 with a time of 4m 14.7s.
dtype: object

In [138]:
typ.columns=[c.replace('.','') for c in typ.columns]
typ.apply(lambda row: pandas_row_mapper(row, rules, "#yy#",[base_english,inflect_english,pytracery_logic]), axis=1)


0             OGI won stage SS1 with a time of 3m 35.3s.
1              NEU took second, 3.7s behind in 3m 39.0s.
2      TÄN took third, a further 0.5s back (4.2s off ...
3                                    MIK ((stagefourth))
4                                     EVA ((stagefifth))
5                                   SOR ((stageseventh))
6                                     BRE ((stageninth))
7                                  LAT ((stageeleventh))
8                                   LAP ((stagetwelfth))
9                                SUN ((stagefourteenth))
10                           LOE ((stagetwenty-seventh))
11                             BLO ((stagethirty-third))
12                            AL  ((stagethirty-fourth))
13                             RAO ((stagethirty-fifth))
14             TÄN took stage SS2 in a time of 4m 19.6s.
15             LAT took second, 2.5s behind in 4m 22.1s.
16     EVA took third, a further 0.5s back (3.0s off ...
17                             

In [127]:
_srs.apply(lambda row: pandas_row_mapper(row, rules, "#yy#",[base_english,inflect_english,pytracery_logic]), axis=1)


56             SOR won stage SS5 in a time of 4m 14.7s.
57            LAT took second, 0.5s behind in 4m 15.2s.
58    BRE took third, a further 0.3s behind (0.8s of...
dtype: object

In [128]:
_srs[_srs['position']==1].apply(lambda row: pandas_row_mapper(row, rules, "#stagewinner_#",[base_english,inflect_english,pytracery_logic]), axis=1)


56    took stage SS5 with a time of 4m 14.7s.
dtype: object

In [111]:
ssd.dbGetSplits(conn,rally,'SS5','RC1').head()

,elapsedDuration,elapsedDurationMs,entryId,splitDateTime,splitDateTimeLocal,splitPointId,splitPointTimeId,stageTimeDuration,stageTimeDurationMs,startDateTime,startDateTimeLocal,stageId,class,code,distance,name,drivercode
0,PT1M8.494S,68494,3044,2018-10-26T13:24:08.494Z,2018-10-26T15:24:08.494+02:00,1650,49603,00:04:18.9000000,258900.0,2018-10-26T13:23:00,2018-10-26T15:23:00+02:00,835,RC1,SS5,7.0,Gandesa 2 (gravel),NEU
1,PT1M9.062S,69062,3040,2018-10-26T13:27:09.062Z,2018-10-26T15:27:09.062+02:00,1650,49604,00:04:18.6000000,258600.0,2018-10-26T13:26:00,2018-10-26T15:26:00+02:00,835,RC1,SS5,7.0,Gandesa 2 (gravel),OGI
2,PT1M8.083S,68083,3047,2018-10-26T13:30:08.083Z,2018-10-26T15:30:08.083+02:00,1650,49605,00:04:16,256000.0,2018-10-26T13:29:00,2018-10-26T15:29:00+02:00,835,RC1,SS5,7.0,Gandesa 2 (gravel),TÄN
3,PT1M8.895S,68895,3048,2018-10-26T13:33:08.895Z,2018-10-26T15:33:08.895+02:00,1650,49606,00:04:31.1000000,271100.0,2018-10-26T13:32:00,2018-10-26T15:32:00+02:00,835,RC1,SS5,7.0,Gandesa 2 (gravel),LAP
4,PT1M8.347S,68347,3046,2018-10-26T13:36:08.347Z,2018-10-26T15:36:08.347+02:00,1650,49607,00:04:15.2000000,255200.0,2018-10-26T13:35:00,2018-10-26T15:35:00+02:00,835,RC1,SS5,7.0,Gandesa 2 (gravel),LAT


In [ ]:
# if pos 1 at last split but not pos 1 at end of stage: "despite being TIME seconds up at the final split.""